In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
#import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB0
from keras.engine import Layer

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [7]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

def shiftImg(img):
    max_translation = 8
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)
    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))

In [8]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=10):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=10):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=10):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=10):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True,clahe=False,mergin=5):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    if clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
    img = img[upper(img,mergin):lower(img,mergin),lefter(img,mergin):righter(img,mergin)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [9]:
def augmix(img):
    img=img.astype(np.float32)
    img1=deepcopy(img)
    img2=deepcopy(img)
    img3=deepcopy(img)
    img1=affine_image(img1)
    img2=affine_image(img2)
    img3=affine_image(img3)
    alpha1=np.random.random()
    alpha2=np.random.random()
    alpha3=np.random.random()
    mix=(img1*alpha1+img2*alpha2*img3*alpha3)/(alpha1+alpha2+alpha3)
    img=img*0.2+mix*0.8
    return img

In [10]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3,clahe=True):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha
        self.clahe=clahe

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training,clahe=self.clahe))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training:
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [augmix(img) for img in imgs]
            imgs = [randomErase(img, prob=True) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [11]:
def getMultiBased():
    model =  EfficientNetB0(weights="imagenet", include_top=False,input_shape=(*size,3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
X=df.values.reshape(-1,137,236)

In [13]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-effb0-m5epoch{epoch:04}.h5")

In [14]:
batch_size = 64

for i in range(4):
    X1=
    train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size,size=size,clahe=False,alpha=32)
    valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size,size=size,clahe=False,alpha=32)
    #strategy = tf.distribute.MirroredStrategy()
    #print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
    #with strategy.scope():
    model = getMultiBased()
    model.summary()
    model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
    print("compiled")
    model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=8, use_multiprocessing=True)
    model.save("multiEfficientB0_mergin5.h5")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 60, 60, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 60, 60, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 60, 60, 32)   0           stem_bn[0][0]                    
____________________________________________________________________________________________

Epoch 1/40
2825/2825 [==============================] - 479s 169ms/step - loss: 2.4523 - dense_2_loss: 3.7292 - dense_3_loss: 1.4660 - dense_4_loss: 0.8847 - dense_2_acc: 0.1955 - dense_3_acc: 0.5115 - dense_4_acc: 0.6925 - val_loss: 1.0737 - val_dense_2_loss: 1.7825 - val_dense_3_loss: 0.4522 - val_dense_4_loss: 0.2978 - val_dense_2_acc: 0.5267 - val_dense_3_acc: 0.8819 - val_dense_4_acc: 0.9118
Epoch 2/40
2825/2825 [==============================] - 454s 161ms/step - loss: 2.0746 - dense_2_loss: 3.1174 - dense_3_loss: 1.2800 - dense_4_loss: 0.7837 - dense_2_acc: 0.3093 - dense_3_acc: 0.5555 - dense_4_acc: 0.7119 - val_loss: 0.9361 - val_dense_2_loss: 1.5309 - val_dense_3_loss: 0.3522 - val_dense_4_loss: 0.3679 - val_dense_2_acc: 0.5871 - val_dense_3_acc: 0.8818 - val_dense_4_acc: 0.8738
Epoch 3/40
2825/2825 [==============================] - 456s 161ms/step - loss: 1.9407 - dense_2_loss: 2.9008 - dense_3_loss: 1.2103 - dense_4_loss: 0.7508 - dense_2_acc: 0.3411 - dense_3_acc: 0.5602 

Epoch 21/40
2825/2825 [==============================] - 454s 161ms/step - loss: 1.4311 - dense_2_loss: 2.0591 - dense_3_loss: 0.9714 - dense_4_loss: 0.6349 - dense_2_acc: 0.4275 - dense_3_acc: 0.5627 - dense_4_acc: 0.7144 - val_loss: 0.4353 - val_dense_2_loss: 0.7624 - val_dense_3_loss: 0.2983 - val_dense_4_loss: 0.2380 - val_dense_2_acc: 0.8600 - val_dense_3_acc: 0.9519 - val_dense_4_acc: 0.9490
Epoch 22/40
2825/2825 [==============================] - 456s 161ms/step - loss: 1.4219 - dense_2_loss: 2.0449 - dense_3_loss: 0.9662 - dense_4_loss: 0.6316 - dense_2_acc: 0.4293 - dense_3_acc: 0.5638 - dense_4_acc: 0.7161 - val_loss: 0.3310 - val_dense_2_loss: 0.6041 - val_dense_3_loss: 0.2751 - val_dense_4_loss: 0.2040 - val_dense_2_acc: 0.8726 - val_dense_3_acc: 0.9498 - val_dense_4_acc: 0.9545
Epoch 23/40
2825/2825 [==============================] - 454s 161ms/step - loss: 1.4100 - dense_2_loss: 2.0249 - dense_3_loss: 0.9611 - dense_4_loss: 0.6292 - dense_2_acc: 0.4289 - dense_3_acc: 0.56

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 456s 162ms/step - loss: 1.3454 - dense_2_loss: 1.9172 - dense_3_loss: 0.9315 - dense_4_loss: 0.6154 - dense_2_acc: 0.4372 - dense_3_acc: 0.5648 - dense_4_acc: 0.7155 - val_loss: 0.4820 - val_dense_2_loss: 0.7823 - val_dense_3_loss: 0.4323 - val_dense_4_loss: 0.2488 - val_dense_2_acc: 0.8596 - val_dense_3_acc: 0.9356 - val_dense_4_acc: 0.9535
Epoch 33/40
  84/2825 [..............................] - ETA: 7:14 - loss: 1.3300 - dense_2_loss: 1.8820 - dense_3_loss: 0.9374 - dense_4_loss: 0.6187 - dense_2_acc: 0.4334 - dense_3_acc: 0.5582 - dense_4_acc: 0.7108

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 455s 161ms/step - loss: 1.3213 - dense_2_loss: 1.8790 - dense_3_loss: 0.9183 - dense_4_loss: 0.6090 - dense_2_acc: 0.4408 - dense_3_acc: 0.5653 - dense_4_acc: 0.7166 - val_loss: 0.5104 - val_dense_2_loss: 0.8192 - val_dense_3_loss: 0.4481 - val_dense_4_loss: 0.2533 - val_dense_2_acc: 0.8528 - val_dense_3_acc: 0.9064 - val_dense_4_acc: 0.9485
Epoch 37/40
 741/2825 [======>.......................] - ETA: 5:28 - loss: 1.3232 - dense_2_loss: 1.8808 - dense_3_loss: 0.9198 - dense_4_loss: 0.6114 - dense_2_acc: 0.4417 - dense_3_acc: 0.5622 - dense_4_acc: 0.7139

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2825/2825 [==============================] - 454s 161ms/step - loss: 1.3059 - dense_2_loss: 1.8520 - dense_3_loss: 0.9142 - dense_4_loss: 0.6053 - dense_2_acc: 0.4443 - dense_3_acc: 0.5653 - dense_4_acc: 0.7162 - val_loss: 0.6359 - val_dense_2_loss: 0.9779 - val_dense_3_loss: 0.5089 - val_dense_4_loss: 0.3020 - val_dense_2_acc: 0.8234 - val_dense_3_acc: 0.8927 - val_dense_4_acc: 0.9236


In [15]:
model.evaluate(valid_gen)

314/314 [==============================] - 25s 78ms/step


[0.635921835899353,
 0.9778517484664917,
 0.5089051723480225,
 0.3019554913043976,
 0.8234415650367737,
 0.892650842666626,
 0.923570990562439]